In [1]:
import gradio as gr
from transformers import pipeline

# Initialize pipelines
chatbot_pipeline = pipeline("text-generation", model="meta-llama/Llama-3.2-8B")  # Verify model name
image_captioning_pipeline = pipeline("image-to-text", model="Salesforce/blip-image-captioning-base")  # Verify model name

# Function for multimodal input handling
def chatbot_or_image(input_type, text_input=None, image_input=None):
    if input_type == "text":
        if not text_input.strip():
            return "Error: Please enter a valid text input for the chatbot."
        response = chatbot_pipeline(text_input, max_length=100, num_return_sequences=1)
        return f"Chatbot Response: {response[0]['generated_text']}"

    elif input_type == "image":
        if image_input is None:
            return "Error: Please upload an image for captioning."
        response = image_captioning_pipeline(image_input)
        return f"Image Caption: {response[0]['generated_text']}"

    return "Error: Invalid input type selected."

# Gradio interface
def update_inputs(input_type):
    if input_type == "text":
        return gr.update(visible=True), gr.update(visible=False)
    elif input_type == "image":
        return gr.update(visible=False), gr.update(visible=True)
    return gr.update(visible=False), gr.update(visible=False)

with gr.Blocks() as iface:
    gr.Markdown("## Multimodal Chatbot")
    gr.Markdown("Chat with text or upload an image for captioning!")

    input_type = gr.Radio(["text", "image"], label="Input Type", value="text")
    text_input = gr.Textbox(label="Text Input", visible=True)
    image_input = gr.Image(type="pil", label="Image Input", visible=False)
    output = gr.Textbox(label="Output")

    input_type.change(
        update_inputs, inputs=[input_type], outputs=[text_input, image_input]
    )

    submit_button = gr.Button("Submit")
    submit_button.click(
        chatbot_or_image,
        inputs=[input_type, text_input, image_input],
        outputs=output,
    )

iface.launch(share=True)


ModuleNotFoundError: No module named 'gradio'